In [5]:
import pandas as pd
from langdetect import detect
from tqdm import tqdm 
tqdm.pandas()
df_dataset = pd.read_csv('public_data/wi_dataset_processed_multilingual.csv')

# # Extract the desired columns
# columns = ["id", "title", "description"]
# df_dataset = df_dataset[columns]


In [20]:
df_dataset['lang'] = df_dataset.progress_apply(lambda x: detect(x["description"]) if (type(x["description"]) == str and len(x["description"]) > 5) else (detect(x["title"]) and print(x["description"])), axis=1)

  1%|▏         | 337/25665 [00:01<02:01, 208.70it/s]

-


  4%|▍         | 1120/25665 [00:05<01:52, 218.05it/s]

-


 13%|█▎        | 3411/25665 [00:16<01:53, 195.76it/s]

...

 78%|███████▊  | 20014/25665 [01:32<00:25, 220.14it/s]

-


 96%|█████████▌| 24647/25665 [01:54<00:04, 212.33it/s]

-


100%|██████████| 25665/25665 [01:58<00:00, 216.03it/s]


In [53]:
from deep_translator import GoogleTranslator
def split_translate_merge(sentence, lang):
    if lang == 'en' or lang == None:
        return sentence
    sentence = sentence.replace('★', '')
    try:
        if len(sentence) > 5000:
            half_length = len(sentence) // 2
            first_half = sentence[:half_length]
            second_half = sentence[half_length:]
            
            translated_first_half = GoogleTranslator(source=lang, target='en').translate(first_half)
            translated_second_half = GoogleTranslator(source=lang, target='en').translate(second_half)
            
            merged_sentence = translated_first_half + translated_second_half
        else:
            merged_sentence = GoogleTranslator(source=lang, target='en').translate(sentence)
        
        return merged_sentence
    except:
        return ''

for index, row in tqdm(df_dataset.iloc[627:].iterrows()):
    df_dataset.at[index, 'translated_description'] = split_translate_merge(row['description'], row['lang'])

25038it [1:33:13,  4.48it/s]


In [61]:
# Step 1: Get the frequency of each value in 'translated_description'
value_counts = df_dataset['translated_description'].value_counts()

# Step 2: Map these frequencies back to the original DataFrame
df_dataset['frequency'] = df_dataset['translated_description'].map(value_counts)

# Step 3: Sort the DataFrame based on the frequency
df_dataset_sorted = df_dataset.sort_values(by='frequency', ascending=False)

# Step 4: Drop the temporary 'frequency' column if not needed
df_dataset_sorted = df_dataset_sorted.drop(columns=['frequency'])

df_dataset_sorted


,id,title,description,lang,translated_description,translated_title,final_description
12931,853649818,Project Administrator (m/w/d) (TRANSFER Planco...,Stellenbeschreibung,de,Job description,Project Administrator (m/f/d) (TRANSFER Planco...,Project Administrator (m/w/d) (TRANSFER Planco...
14533,788289588,Java Full Stack Developer (m/w/d) (softwareXpe...,Stellenbeschreibung,de,Job description,Java Full Stack Developer (m/f/d) (softwareXpe...,Java Full Stack Developer (m/w/d) (softwareXpe...
11729,849226346,Network Engineer IP Operations (m/w/d) (Magent...,Stellenbeschreibung,de,Job description,Network Engineer IP Operations (m/f/d) (Magent...,Network Engineer IP Operations (m/w/d) (Magent...
19710,867443453,Head of Corporate Security (f/m/d) (Bank Austria),Stellenbeschreibung,de,Job description,Head of Corporate Security (f/m/d) (Bank Austria),Head of Corporate Security (f/m/d) (Bank Austria)
13629,775084837,Pflegefachkraft (m/w) (Pflegeheim Margot),Stellenbeschreibung,de,Job description,Nursing professional (m/f) (Margot Nursing Home),Pflegefachkraft (m/w) (Pflegeheim Margot)
...,...,...,...,...,...,...,...
8685,864113304,Error during processing.,The page that you are trying to access cannot ...,en,The page that you are trying to access cannot ...,Error during processing.,Error during processing. The page that you are...
8684,789341328,Warehouse Picker (Nights),Warehouse Picker (Nights) Market 36 Recruitmen...,en,Warehouse Picker (Nights) Market 36 Recruitmen...,Warehouse Picker (Nights),Warehouse Picker (Nights) Warehouse Picker (Ni...
8683,754578581,Hydaulic Modeller / Hydrologist,Hydaulic Modeller / Hydrologist -Required for ...,en,Hydaulic Modeller / Hydrologist -Required for ...,Hydaulic Modeller / Hydrologist,Hydaulic Modeller / Hydrologist Hydaulic Model...
8682,798914884,Fenergo Product Consultant,Responsibilities Responsibilities: * design cl...,en,Responsibilities Responsibilities: * design cl...,Fenergo Product Consultant,Fenergo Product Consultant Responsibilities Re...


In [76]:
empty_string_count = df_dataset["translated_description"].isna().sum()
print(empty_string_count)

2


In [65]:
for index, row in tqdm(df_dataset.iterrows()):
    df_dataset.at[index, 'translated_title'] = split_translate_merge(row['title'], row['lang'])

25665it [1:06:39,  6.42it/s]


In [32]:
df_dataset[df_dataset['translated_description'].isnull()]
# short_sentences = df_dataset[df_dataset['translated_description'].str.len() < 50]

,id,title,description,lang,translated_description,frequency,translated_title
627,745609437,"Специалист оперативна дейност, Пловдив, Пловди...",Разглеждания: 1 012 13.11.2020 ★★★★★ Специалис...,bg,NaN,39.0,"Operations specialist, Plovdiv, Plovdiv, job a..."
1964,724138551,"Специалист производствено ПЛАНИРАНЕ, Плевен, о...","Разглеждания: 343 13.10.2020, Ref.#: PPS ★ Спе...",bg,NaN,39.0,"Production PLANNING specialist, Pleven, job ad..."
2567,781522612,Πανεπιστήμιο Κύπρου - Εργαστήριο Φωτοβολταϊκής...,Πανεπιστήμιο Κύπρου Job Overview Expiration da...,el,NaN,39.0,University of Cyprus - Photovoltaic Technology...
3380,800291956,Assistant marketing et commercial en alternanc...,NaN,NaN,NaN,NaN,Assistant marketing et commercial en alternanc...
3734,880387533,"Горски стражар, Огняново, обява за работа от Т...",обратно Разглеждания: 07.09.2021 Горски стража...,bg,NaN,39.0,"Forest guard, Ognyanovo, job advertisement fro..."
4871,786367021,"- Машинен оператор / линия за довършване, Софи...","Разглеждания: *** **********, Ref.#: MOF ★★★★★...",bg,NaN,39.0,"- Machine operator / finishing line, Sofia, jo..."
4979,848168683,Υπάλληλοι Καταστήματος (μερική απασχόληση) - Ά...,Υπάλληλοι Καταστήματος (μερική απασχόληση) στο...,el,NaN,39.0,Store Clerks (part-time) - Agios Stefanos
6425,741866124,"ХИГИЕНИСТ/КА АДМИНИСТРАТИВНА СГРАДА, Стара Заг...",Разглеждания: 474 09.11.2020 ХИГИЕНИСТ/КА АДМИ...,bg,NaN,39.0,"HYGIENIST ADMINISTRATIVE BUILDING, Stara Zagor..."
6885,825377375,Σύμβουλος Πωλήσεων Κινητής Τηλεφωνίας & Συνδέσ...,Εργασία στην Ελλάδα Menu Επιστροφή Αγγελίες Κω...,el,NaN,39.0,Mobile Telephony & Connections Sales Consultan...
8183,732701981,"Монтажник, ЕЛЕКТРООБОРУДВАНЕ, Стряма, обява за...","Разглеждания: 57 28.10.2020 ★★★ МОНТАЖНИК, ЕЛЕ...",bg,NaN,39.0,"Installer, ELECTRICAL EQUIPMENT, Stryama, job ..."


In [28]:
df_dataset[df_dataset['translated_description'].isnull()]

,id,title,description,lang,translated_description,frequency,translated_title
627,745609437,"Специалист оперативна дейност, Пловдив, Пловди...",Разглеждания: 1 012 13.11.2020 ★★★★★ Специалис...,bg,NaN,NaN,"Operations specialist, Plovdiv, Plovdiv, job a..."
1964,724138551,"Специалист производствено ПЛАНИРАНЕ, Плевен, о...","Разглеждания: 343 13.10.2020, Ref.#: PPS ★ Спе...",bg,NaN,NaN,"Production PLANNING specialist, Pleven, job ad..."
2567,781522612,Πανεπιστήμιο Κύπρου - Εργαστήριο Φωτοβολταϊκής...,Πανεπιστήμιο Κύπρου Job Overview Expiration da...,el,NaN,NaN,University of Cyprus - Photovoltaic Technology...
3380,800291956,Assistant marketing et commercial en alternanc...,NaN,NaN,NaN,NaN,Assistant marketing et commercial en alternanc...
3734,880387533,"Горски стражар, Огняново, обява за работа от Т...",обратно Разглеждания: 07.09.2021 Горски стража...,bg,NaN,NaN,"Forest guard, Ognyanovo, job advertisement fro..."
4871,786367021,"- Машинен оператор / линия за довършване, Софи...","Разглеждания: *** **********, Ref.#: MOF ★★★★★...",bg,NaN,NaN,"- Machine operator / finishing line, Sofia, jo..."
4979,848168683,Υπάλληλοι Καταστήματος (μερική απασχόληση) - Ά...,Υπάλληλοι Καταστήματος (μερική απασχόληση) στο...,el,NaN,NaN,Store Clerks (part-time) - Agios Stefanos
6425,741866124,"ХИГИЕНИСТ/КА АДМИНИСТРАТИВНА СГРАДА, Стара Заг...",Разглеждания: 474 09.11.2020 ХИГИЕНИСТ/КА АДМИ...,bg,NaN,NaN,"HYGIENIST ADMINISTRATIVE BUILDING, Stara Zagor..."
6885,825377375,Σύμβουλος Πωλήσεων Κινητής Τηλεφωνίας & Συνδέσ...,Εργασία στην Ελλάδα Menu Επιστροφή Αγγελίες Κω...,el,NaN,NaN,Mobile Telephony & Connections Sales Consultan...
8183,732701981,"Монтажник, ЕЛЕКТРООБОРУДВАНЕ, Стряма, обява за...","Разглеждания: 57 28.10.2020 ★★★ МОНТАЖНИК, ЕЛЕ...",bg,NaN,NaN,"Installer, ELECTRICAL EQUIPMENT, Stryama, job ..."


In [34]:
null_indices = df_dataset[df_dataset['translated_description'].isnull()].index
null_indices

Index([  627,  1964,  2567,  3380,  3734,  4871,  4979,  6425,  6885,  8183,
        8417,  9191,  9729,  9914, 10103, 10216, 11066, 11292, 11359, 11720,
       11979, 12595, 12772, 14041, 14048, 15064, 15225, 16398, 17076, 17152,
       17231, 17496, 20278, 20442, 20653, 21620, 21889, 22436, 23284, 24742,
       25219],
      dtype='int64')

In [40]:
df_dataset.loc[627, 'description']

'Разглеждания: 1 012 13.11.2020 ★★★★★ Специалист оперативна дейност, Пловдив, UniCredit Bulbank AD  \uf451Вижте визитка на компанията Месторабота Пловдив; Постоянна работа; Пълно работно време Запази обявата в бележника   Известие за нови обяви   Принтирай   Проблем с обявата Специалист оперативна дейност, Пловдив Описание Всеки от нас непрекъснато търси най-доброто място за работа. Място, където идеите ни са наистина от значение. Ние в УниКредит инвестираме в твоя талант, защото за нас хората са най-големият ни актив, който допринася за общността ни с разнообразния си опит, умения и светоглед. Ние подпомагаме хората ни да растат като разкриваме уникалния им потенциал и инвестираме в тяхното професионално и личностно развитие. Стани част от бъдещето, което градим всички заедно в една международна група, където Хората са много повече от служители и всеки съществено допринася за споделения ни успех. Ти можеш да реализираш плановете си, да оставиш следа, да правиш важните неща. Основни за

In [42]:
ts.translate_text(df_dataset.loc[627, 'description'], from_language='bg', to_language='en', translator='google')

'The examinations: 1 012 13.11.2020 ★★★★ 7 Specialist Operational Activity, Plovdiv, UniCredit Bulbank AD \uf451 View business card of the company location Plovdiv; Permanent work; Full-time keep the advertisement in the notebook notice of new advertisement Print a problem with the ad specialist Operational Activity, Plovdiv Description Each of us is constantly looking for the best place to work. A place where our ideas are really important. We at UniCredit invest in your talent because for us people are our biggest asset, which contributes to our community with its diverse experience, skills and worldview. We help our people grow by revealing their unique potential and investing in their professional and personal development. Become a part of the future that we all build together in an international group, where people are much more than employees and everyone significantly contributes to our shared success. You can realize your plans, leave a mark, do the important things. Financial 

In [43]:
import translators as ts
def split_translate_merge(sentence, lang):
    if lang == 'en' or lang == None:
        return sentence
    try:
        if len(sentence) > 5000:
            half_length = len(sentence) // 2
            first_half = sentence[:half_length]
            second_half = sentence[half_length:]
            
            translated_first_half = ts.translate_text(first_half, from_language=lang, to_language='en', translator='google')
            translated_second_half = ts.translate_text(second_half, from_language=lang, to_language='en', translator='google')
            
            merged_sentence = translated_first_half + translated_second_half
        else:
            merged_sentence = ts.translate_text(sentence, from_language=lang, to_language='en', translator='google')
        print(merged_sentence)
        return merged_sentence
    except:
        return ''
# null_indices = df_dataset[df_dataset['translated_description'].isnull()].index

# Re-translate the rows where 'translated_description' is null
df_dataset.loc[null_indices, 'translated_description'] = \
    df_dataset.loc[null_indices].progress_apply(
        lambda x: split_translate_merge(x['description'], x['lang']), axis=1
    )

  0%|          | 0/41 [00:00<?, ?it/s]

The examinations: 1 012 13.11.2020 ★★★★ 7 Specialist Operational Activity, Plovdiv, UniCredit Bulbank AD  View business card of the company location Plovdiv; Permanent work; Full-time keep the advertisement in the notebook notice of new advertisement Print a problem with the ad specialist Operational Activity, Plovdiv Description Each of us is constantly looking for the best place to work. A place where our ideas are really important. We at UniCredit invest in your talent because for us people are our biggest asset, which contributes to our community with its diverse experience, skills and worldview. We help our people grow by revealing their unique potential and investing in their professional and personal development. Become a part of the future that we all build together in an international group, where people are much more than employees and everyone significantly contributes to our shared success. You can realize your plans, leave a mark, do the important things. Financial and no

  7%|▋         | 3/41 [00:02<00:27,  1.36it/s]

Consideration: 343 13.10.10.2020, ref.#: PPS ★ Specialist production planning, Leoni Bulgaria EOOD PLACE PLEVEN; Permanent work; Full -time; Suitable for candidates with small or non -experienced candidates in the notebook notice for new advertisement Prints a problem with the ad specialist production planning Leoni is a global supplier of products, solutions and services for energy management and data in the automotive sector and other industries. The range of products includes cables, optical fibers, standardized cables, special cables and installation systems, as well as smart products and services. As an innovative partner and solution supplier, Leoni maintains its customers with clear development and systematic experience. The company, which was registered with the German MDAX, has more than 95,000 people in 32 countries and has realized consolidated sales of 4.8 billion euros in 2019. Leoni started its activity in Bulgaria in 2018 and are already in the phase of the phase of Cons

 10%|▉         | 4/41 [00:05<00:54,  1.48s/it]

...

Job offer ********* more convenient with the application ********** Log in, create an account for companies, main search for offers saved account, use a browser that we do not support anymore. Our site may not work properly. Choose one of the browsers we recommend. Ok Work Wrocław Wałbrzych Legnica Jelenia Góra Lubin Głogów Warsaw Kraków Łódź IT - software development Business analysis Programming Testing Apply Apply Print Share the junior quality analyst - software tester Acturis Poland Sp. z o.o.o company Legnicka 48F, Wrocław, Lower Silesia valid for 2 days until: 18 Apr 2021 Recruitment Remote employment contract full -time Junior Specialist (Junior) Acturis Poland Sp. z o.o. Legnicka 48F Wrocław see on the map technologies we use welcome Redmine Testrail Oracle SQL Developer Selenium Alto XMLSPY Microsoft Visual Basic for Applications Operating System Your scope of duties review of technical documentation and creating test documentation (scenarios, test cases) Creating and maintai

100%|██████████| 41/41 [01:05<00:00,  1.75s/it]

On the back: 03.09.2021 junior lawyer/legal counsel, Invictus Consult EOOD Location Sofia; Permanent work; Full -time; Salary from 1500 to 1800 bGN (net) kept the advertisement in the notebook of new advertisement Problem Problem with the announcement junior lawyer/legal counsel Consultation House "Invictus Consult" Ltd. specializes in legal services as well as of individuals working In close cooperation with a law firm. In connection with the expansion of the activity, we are looking for a responsible and ambitious person for the position of junior lawyer/legal counsel: professional requirements: - excellent legal training; - acquired legal capacity; - Attorney's legal capacity will be considered an advantage; - experience in serving corporate clients will be considered an advantage; - proficiency in English; - execution of assigned assignments on time, skills for prioritizing the tasks; - work with legal information portals; Other requirements: - working with the Word and Excel offic

100%|██████████| 41/41 [01:07<00:00,  1.64s/it]

Announcement Agency for Social Assistance, Sofia, 2 Triaditsa Str., On the grounds of Art. For the position of Chief Psychologist, Department of Child Protection Department at the Social Assistance Directorate-Slatina, region. Sofia-city II. Requirements for the post of Chief Expert Psychologist in the Child Protection Department: 1. Minimum: · Bachelor-Bachelor · Minimum professional experience-2 years and/or IV junior rank 2. Additional: · Analytical competence; · Orientation to results; · Teamwork; · Communication competence; · Focus to the client; · Professional competence; · Digital competence. III. The competition will be conducted by resolving a test and conducting an interview. IV. Applicants submit a written application for participation in the competition (Annex No. 3 to Article 17, Paragraph 2 of the CCPCSL). The following documents should be attached to the application: 1. a declaration by the person that: (a) has reached the age of majority and is: · a Bulgarian citizen; ·

In [65]:
df_dataset

,id,title,description,lang,translated_description,frequency,translated_title,final_description
0,872828466,Panel & Paint Technician,Panel & Paint Technician required in Colcheste...,en,Panel & Paint Technician required in Colcheste...,1,Panel & Paint Technician,Panel & Paint Technician Panel & Paint Technic...
1,839465958,"Lärare i slöjd och teknik för årkurs 7-9, Ljun...",Sista ansökningsdatum: 1 juni 2021 Referensnum...,sv,"Application deadline: June 1, 2021 Reference n...",1,Teacher in crafts and technology for grades 7-...,"Lärare i slöjd och teknik för årkurs 7-9, Ljun..."
2,857077872,Consultants in Emergency Medicine - Doughiska,"The Galway Clinic is a leading 146 bed, state ...",en,"The Galway Clinic is a leading 146 bed, state ...",1,Consultants in Emergency Medicine - Doughiska,Consultants in Emergency Medicine - Doughiska ...
3,801801567,Senior IT Support Engineers,"My Client, who has been continually growing th...",en,"My Client, who has been continually growing th...",1,Senior IT Support Engineers,"Senior IT Support Engineers My Client, who has..."
4,855162927,Commercial Sales Representatives,"Jobbtitel: ""Commercial Sales Representatives"" ...",en,"Jobbtitel: ""Commercial Sales Representatives"" ...",1,Commercial Sales Representatives,"Commercial Sales Representatives Jobbtitel: ""C..."
...,...,...,...,...,...,...,...,...
25660,862998979,Продавач-консултант в шоурум Gallerato,Описание и изисквания: Gallerato е фирма с дъл...,bg,Description and requirements: Gallerato is a c...,1,Gallerato showroom sales consultant,Продавач-консултант в шоурум Gallerato Descrip...
25661,793143661,Pedestrian Marshall,Pedestrial Marshall / Yardman Do you have a DB...,en,Pedestrial Marshall / Yardman Do you have a DB...,1,Pedestrian Marshall,Pedestrian Marshall Pedestrial Marshall / Yard...
25662,725881734,Unity Gameplay Developer (f/m/d),Unity Gameplay Developer (f/m/d) - Job bei Sun...,en,Unity Gameplay Developer (f/m/d) - Job bei Sun...,1,Unity Gameplay Developer (f/m/d),Unity Gameplay Developer (f/m/d) Unity Gamepla...
25663,880528881,Carpenter 4505717 | careers4a.com,"What: job title, keywords or skills Where:...",en,"What: job title, keywords or skills Where:...",1,Carpenter 4505717 | careers4a.com,Carpenter 4505717 | careers4a.com What: jo...


In [81]:
unwanted_keywords = [
    "You will now be redirected", 
    "reCAPTCHA check page reCAPTCHA check page", 
    "We use cookies to ensure", 
    "Your browser does not support", 
    "Vacancies & jobs in the Joblift job search We have all the jobs. Therefore, you only need one page for your job search: You will now be redirected to our partner. If you are not redirected within the next few seconds, please click here",
    "Job - Job ads - Job offers - CV Market To continue browsing our portal, contact +370 5 219 0032",
    "Home Job offer Are you looking for people? Login Register Service price list © 2021 Zoznam, s.r.o. All rights reserved.",
    "Send CV In the next step, you will be able to choose"
]
df_dataset['description'] = df_dataset['description'].fillna('')
df_dataset['translated_description'] = df_dataset['translated_description'].fillna('')
df_dataset["final_description"] = df_dataset.progress_apply(
    lambda x: x["translated_title"] 
    if any(keyword in x['translated_description'] for keyword in unwanted_keywords) or len(x['translated_description']) < 50
    else x["translated_title"] + " " + x["translated_description"], 
    axis=1
)


100%|██████████| 25665/25665 [00:00<00:00, 38959.63it/s]


In [82]:
df_dataset.to_csv('public_data/wi_dataset_processed_en.csv', index=False)

In [6]:
df_dataset

,id,title,description,lang,translated_description,frequency,translated_title,final_description
0,872828466,Panel & Paint Technician,Panel & Paint Technician required in Colcheste...,en,Panel & Paint Technician required in Colcheste...,1,Panel & Paint Technician,Panel & Paint Technician Panel & Paint Technic...
1,839465958,"Lärare i slöjd och teknik för årkurs 7-9, Ljun...",Sista ansökningsdatum: 1 juni 2021 Referensnum...,sv,"Application deadline: June 1, 2021 Reference n...",1,Teacher in crafts and technology for grades 7-...,"Lärare i slöjd och teknik för årkurs 7-9, Ljun..."
2,857077872,Consultants in Emergency Medicine - Doughiska,"The Galway Clinic is a leading 146 bed, state ...",en,"The Galway Clinic is a leading 146 bed, state ...",1,Consultants in Emergency Medicine - Doughiska,Consultants in Emergency Medicine - Doughiska ...
3,801801567,Senior IT Support Engineers,"My Client, who has been continually growing th...",en,"My Client, who has been continually growing th...",1,Senior IT Support Engineers,"Senior IT Support Engineers My Client, who has..."
4,855162927,Commercial Sales Representatives,"Jobbtitel: ""Commercial Sales Representatives"" ...",en,"Jobbtitel: ""Commercial Sales Representatives"" ...",1,Commercial Sales Representatives,"Commercial Sales Representatives Jobbtitel: ""C..."
...,...,...,...,...,...,...,...,...
25660,862998979,Продавач-консултант в шоурум Gallerato,Описание и изисквания: Gallerato е фирма с дъл...,bg,Description and requirements: Gallerato is a c...,1,Gallerato showroom sales consultant,Продавач-консултант в шоурум Gallerato Описани...
25661,793143661,Pedestrian Marshall,Pedestrial Marshall / Yardman Do you have a DB...,en,Pedestrial Marshall / Yardman Do you have a DB...,1,Pedestrian Marshall,Pedestrian Marshall Pedestrial Marshall / Yard...
25662,725881734,Unity Gameplay Developer (f/m/d),Unity Gameplay Developer (f/m/d) - Job bei Sun...,en,Unity Gameplay Developer (f/m/d) - Job bei Sun...,1,Unity Gameplay Developer (f/m/d),Unity Gameplay Developer (f/m/d) Unity Gamepla...
25663,880528881,Carpenter 4505717 | careers4a.com,"What: job title, keywords or skills Where:...",en,"What: job title, keywords or skills Where:...",1,Carpenter 4505717 | careers4a.com,Carpenter 4505717 | careers4a.com What: jo...


In [7]:
out_columns = ["id", "title", "lang", "description", "final_description"]
df_dataset[out_columns].to_csv('public_data/wi_dataset_processed_multilingual.csv', index=False)